In [0]:
import pandas as pd

In [0]:
thing = pd.read_csv("/Workspace/sid-v2/regression/lrm/insurance.csv")
thing.head()

In [0]:
thing.info()


In [0]:
thing.describe()

In [0]:
thing.isnull().sum()

In [0]:
import torch
import torch.nn as nn
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch.optim as optim


In [0]:
traindata, testdata = train_test_split(thing, test_size=0.25, train_size=0.75, random_state=420)


In [0]:
traindata

In [0]:
testdata

In [0]:
thing

In [0]:
0.75 * 1338

In [0]:
labelencodifier = {}
for i in ["sex", "smoker", "region"]:
    le = LabelEncoder()
    traindata[i] = le.fit_transform(traindata[i])  
    testdata[i] = le.transform(testdata[i])        
    labelencodifier[i] = le

In [0]:
class modell(nn.Module):
    def __init__(self, inputdata):  
        super(modell, self).__init__() 
        self.network = nn.Sequential(
            nn.Linear(inputdata, 64),
            nn.ReLU(),  
            nn.Linear(64, 128),
            nn.ReLU(),  
            nn.Linear(128, 1)
        )
    
    def forward(self, x):  
        return self.network(x)

In [0]:
X_train = traindata.drop('charges', axis=1).values 
y_train = traindata['charges'].values  
X_test = testdata.drop('charges', axis=1).values  
y_test = testdata['charges'].values  

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)  
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)  
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)  
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)  

print(f"Training features shape: {X_train_tensor.shape}")
print(f"Training target shape: {y_train_tensor.shape}")
print(f"Test features shape: {X_test_tensor.shape}")
print(f"Test target shape: {y_test_tensor.shape}")

input_size = X_train_tensor.shape[1]  
model = modell(input_size)  
print(f"Model input size: {input_size}")
print(f"Model architecture:\n{model}")

criterion = nn.MSELoss()
optimiser = optim.Adam(model.parameters(), lr=0.001)

epochs = 10000
for epoch in range(epochs):
    model.train()
    optimiser.zero_grad()
    
    pred = model(X_train_tensor)  
    loss = criterion(pred, y_train_tensor)  
    
    loss.backward()
    optimiser.step()
    
    if epoch % 100 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

model.eval()
with torch.no_grad():
    test_pred = model(X_test_tensor)
    test_loss = criterion(test_pred, y_test_tensor)
    print(f"\nTest Loss: {test_loss.item():.4f}")

In [0]:
criterion = nn.MSELoss()
optimiser = optim.Adam(model.parameters(), lr=0.12)

In [0]:
import torch
from torch.utils.data import Dataset, DataLoader

In [0]:
import torch
from torch.utils.data import Dataset, DataLoader

class InsuranceDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        features = torch.tensor(self.X[idx], dtype=torch.float32)
        target = torch.tensor(self.y[idx], dtype=torch.float32)  # Remove .values
        return features, target

dataset = InsuranceDataset(X_train, y_train)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [0]:
for batch_idx, (features, targets) in enumerate(dataloader):
    print(f"Batch {batch_idx+1} :")
    print("Features : ", features.shape)